### 기본 DB

In [49]:
# 기본DB 7차 / 8차

import pandas as pd
import olefile
import zlib
import struct

# ns_7

def get_hwp_text(filename) :
    f = olefile.OleFileIO(filename)
    dirs = f.listdir()

    # HWP 파일 검증
    if ["FileHeader"] not in dirs or \
       ["\x05HwpSummaryInformation"] not in dirs :
        raise Exception("Not Valid HWP.")

    # 문서 포맷 압축 여부 확인
    header = f.openstream("FileHeader")
    header_data = header.read()
    is_compressed = (header_data[36] & 1) == 1

    # Body Sections 불러오기
    nums = []
    for d in dirs :
        if d[0] == "BodyText" :
            nums.append(int(d[1][len("Section"):]))
    sections = ["BodyText/Section" + str(x) for x in sorted(nums)]

    # 전체 text 추출
    text = ""
    for section in sections :
        bodytext = f.openstream(section)
        data = bodytext.read()
        if is_compressed :
            unpacked_data = zlib.decompress(data, -15)
        else:
            unpacked_data = data
    
        # 각 Section 내 text 추출    
        section_text = ""
        i = 0
        size = len(unpacked_data)
        while i < size :
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type in [67] and header != 304088131 and header != 18879555 and  header != 18875459 and  header != 35652675 and  header != 23069763 and  header != 52429891 and  header != 20972611 :
                rec_data = unpacked_data[i + 4 : i + 4 + rec_len]
                print(header)
                print(rec_data.decode('UTF-16'))
                section_text += rec_data.decode('UTF-16')
                section_text += "\n"

            i += 4 + rec_len
        
        text += section_text
        text += "\n"

    return text

In [50]:
# ns_8

def get_hwp_text2(filename) :
    f = olefile.OleFileIO(filename)
    dirs = f.listdir()

    # HWP 파일 검증
    if ["FileHeader"] not in dirs or \
       ["\x05HwpSummaryInformation"] not in dirs :
        raise Exception("Not Valid HWP.")

    # 문서 포맷 압축 여부 확인
    header = f.openstream("FileHeader")
    header_data = header.read()
    is_compressed = (header_data[36] & 1) == 1

    # Body Sections 불러오기
    nums = []
    for d in dirs :
        if d[0] == "BodyText" :
            nums.append(int(d[1][len("Section"):]))
    sections = ["BodyText/Section" + str(x) for x in sorted(nums)]

    # 전체 text 추출
    text = ""
    for section in sections :
        bodytext = f.openstream(section)
        data = bodytext.read()
        if is_compressed :
            unpacked_data = zlib.decompress(data, -15)
        else:
            unpacked_data = data
    
        # 각 Section 내 text 추출    
        section_text = ""
        i = 0
        size = len(unpacked_data)
        while i < size :
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type in [67] and header != 69207107 and header != 18875459 and header != 35652675 :
                rec_data = unpacked_data[i + 4 : i + 4 + rec_len]
                print(header)
                print(rec_data.decode('UTF-16'))
                section_text += rec_data.decode('UTF-16')
                section_text += "\n"

            i += 4 + rec_len
        
        text += section_text
        text += "\n"

    return text

In [ ]:
# 한자가 적힌 변수

# ns_7
'''
304088131
18879555
18875459
35652675
23069763
52429891
20972611
'''

# 8차
'''
69207107
18875459
35652675
'''

In [51]:
# HWP 파일 불러오기

기본DB_7차 = get_hwp_text('data/ns_7.hwp')
기본DB_8차 = get_hwp_text2('data/ns_8.hwp')

18877507
漠杳    
18877507
汤捯    
12586051
mod_d
20974659
최종 DB 수정일
6294595
ID
14683203
개인 아이디
14683203
ID_fam
14683203
가구 아이디
10488899
year
10488899
조사연도
14683203
region
14683203
17개 시도
14683203
town_t
16780355
동/읍면 구분
12586051
apt_t
14683203
아파트 구분
8391747
psu
12586051
조사구번호
8391747
sex
6294595
성별
8391747
age
8391747
만나이
20974659
age_month
6294595
월령
10488899
incm
25168963
소득 4분위수(개인)
16780355
ho_incm
25168963
소득 4분위수(가구)
12586051
incm5
25168963
소득 5분위수(개인)
18877507
ho_incm5
25168963
소득 5분위수(가구)
8391747
edu
25168963
교육수준 재분류 코드
10488899
occp
46140483
직업재분류 및 실업/비경제활동상태 코드
12586051
wt_hs
18877507
가구조사 가중치
18877507
wt_itvex
29363267
건강설문-검진조사 가중치
14683203
wt_pft
37751875
폐기능검사 가중치(만40세이상)
12586051
wt_hm
37751875
중금속검사 가중치(만10세이상)
12586051
wt_vt
37751875
비타민검사 가중치(만10세이상)
12586051
wt_nn
37751875
NNAL검사 가중치(만6세이상)
14683203
wt_ex1
33557571
안검사 가중치(만40세이상)
14683203
wt_ntr
18877507
영양조사 가중치
14683203
wt_tot
44043331
건강설문-검진조사 & 영양조사 가중치
16780355
wt_pfhm
54529091
폐기능검사 & 중금속검사 가중치(만40세이상)
1

In [52]:
# ''\r\n' split

기본DB_7차 = 기본DB_7차.split('\r\n')
기본DB_8차 = 기본DB_8차.split('\r\n')

In [53]:
기본DB_7차 = 기본DB_7차[2:] # 한자제거

In [54]:
## 딕셔너리 형태로 생성

기본DB_7차_dic = dict()
for i in range(0, len(기본DB_7차)-1,2) :
    기본DB_7차_dic[기본DB_7차[i]] = 기본DB_7차[i+1] 

기본DB_8차_dic = dict()
for i in range(0, len(기본DB_8차)-1,2) :
    기본DB_8차_dic[기본DB_8차[i]] = 기본DB_8차[i+1]


In [57]:
# 데이터프레임 생성

기본DB_7차_df = pd.DataFrame(list(기본DB_7차_dic.items()),
                   columns = ['변수명','변수설명'])

기본DB_8차_df = pd.DataFrame(list(기본DB_8차_dic.items()),
                   columns = ['변수명','변수설명'])

In [58]:
# 변수명 대문자화

upper_7 = []
for i in 기본DB_7차_df.loc[ : , '변수명'] :
    upper_7.append(i.upper())

upper_8 = []
for i in 기본DB_8차_df.loc[ : , '변수명'] :
    upper_8.append(i.upper())

기본DB_7차_df['변수명'] = upper_7
기본DB_8차_df['변수명'] = upper_8

In [61]:
# 데이터 프레임 csv 파일로 저장

기본DB_7차_df.to_csv('data/기본DB_7.csv', encoding = 'euc-kr')
기본DB_8차_df.to_csv('data/기본DB_8.csv', encoding = 'euc-kr')